In [58]:
import datasets
import utils
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
import GPflow
import GPy

In [5]:
dset = datasets.datasets(exclude_labels=False)["LetterRecognition"][0]
test_mask = np.random.rand(len(dset)) < 0.2
y_train = dset["lettr"].values[~test_mask]
X_train = dset.drop(["lettr"], axis=1).values[~test_mask]

print(X_train.dtype, y_train.dtype)
Xm, Xs = X_train.mean(axis=0), X_train.std(axis=0)

X_train = (X_train-Xm)/Xs
y_test = dset["lettr"].values[test_mask]
X_test = (dset.values[test_mask, 1:]-Xm)/Xs


+++ Importing BostonHousing
+++ Importing BreastCancer
+++ Importing Servo
+++ Importing Shuttle
+++ Importing Ionosphere
V2 ; must have more than 1 possible value
+++ Importing Soybean
+++ Importing LetterRecognition
float64 int32


In [12]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1,
                           max_features=int(np.floor(X_train.shape[1]**.5)),
                           bootstrap=False,
                           min_samples_split=5)
rf.fit(X_train, y_train)
train_perf = rf.score(X_train, y_train)
test_perf = rf.score(X_test, y_test)
print(accuracy_score(rf.predict(X_train), y_train))
print(accuracy_score(rf.predict(X_test), y_test))
print("Test: {:f}, Training: {:f}".format(test_perf, train_perf))

Test: 0.968479, Training: 0.999937


In [3]:
def r_squared(y_true, y):
    u = np.sum((y-y_true)**2)
    v = np.sum((y_true - np.mean(y_true))**2)
    return 1 - u/v

def 

In [17]:
import selu
import denoising_ae as dae
import imp
imp.reload(dae)
imp.reload(selu)

tf_float = tf.float32
tf.reset_default_graph()

nn_X = tf.placeholder(tf_float, shape=[None, X_train.shape[1]], name="X")
nn_y = tf.placeholder(tf.int32, shape=[None], name="y")
nn_kp = tf.placeholder(tf_float, shape=[], name="keep_prob")
nn_lr = tf.placeholder(tf_float, shape=[], name="learning_rate")


nn_preds = dae.FC_net(nn_X, [(512, selu.nlin)]*4 + [(y_train.max()+1, None)], selu.initializer, keep_prob=nn_kp)
nn_loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=nn_preds, labels=nn_y))
nn_labels = tf.argmax(nn_preds, axis=1)
nn_train = tf.train.GradientDescentOptimizer(nn_lr).minimize(nn_loss)

In [18]:
from tqdm import tqdm
import itertools

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

batch_size = 128

num_batches = int(np.ceil(X_train.shape[0] / batch_size))

pbar = tqdm(itertools.count(0))
for step in pbar:
    i = step % num_batches
    # Populate feed_dict; duplicated code for num and cat
    s = slice(batch_size * i, batch_size * (i + 1))
    feed_dict = {nn_kp: 0.95,
                 nn_lr: 0.001 if step < 800 else 0.0001,
                 nn_X: X_train[s],
                 nn_y: y_train[s],
                }
    del i
    
    if step % 100 == 0:
        _, l, y = sess.run([nn_train, nn_loss, nn_labels], feed_dict)
        d = "Loss: {} ".format(l)
        d += "Train: {} ".format(accuracy_score(y_train[s], y))
        y = sess.run(nn_labels, {nn_kp: 1.0,
                                 nn_X: X_test,
                                 nn_y: y_test})
        d += "Test: {} ".format(accuracy_score(y_test, y))
        pbar.set_description(d)
    else:
        sess.run(nn_train, feed_dict)


0it [00:00, ?it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 2it [00:00, 17.82it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 15it [00:00, 24.00it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 28it [00:00, 31.66it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 41it [00:00, 40.81it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 54it [00:00, 50.93it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 64it [00:00, 59.67it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 75it [00:00, 62.64it/s]
Loss: 3.73152494430542 Train: 0.046875 Test: 0.055100521221146684 : 88it [00:00, 73.81it/s]
Loss: 2.016108989715576 Train: 0.5390625 Test: 0.46487962273517003 : 101it [00:01, 78.12it/s]
Loss: 2.016108989715576 Train: 0.5390625 Test: 0.46487962273517003 : 114it [00:01, 88.40it/s]
Loss: 2.016108989715576 Train: 0.5390625 Test: 0.46487962

KeyboardInterrupt: 

In [41]:
tf.summary.FileWriter("faceoff/", graph=sess.graph)

Loss: 0.055597126483917236 Train: 0.9541863795146673 Test: 0.9235167656130201 : 3687it [02:48, 21.89it/s]

In [23]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
import time

# Fit KernelRidge with parameter selection based on 5-fold cross validation
param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [RBF(l)
                         for l in np.logspace(-2, 2, 10)]}
kr = GridSearchCV(KernelRidge(), cv=5, param_grid=param_grid)
stime = time.time()
kr.fit(X_train[:100], y_train[:100])
print("Time for KRR fitting: %.3f" % (time.time() - stime))

Time for KRR fitting: 0.549


In [18]:
Z = np.random.rand(1000, X_train.shape[1])
rbf = GPflow.kernels.RBF(X_train.shape[1], variance=1, ARD=True)
m = GPflow.sgpr.SGPR(X_train, y_train[:,None], kern=rbf, Z=Z)
def logger(x):
    if (logger.i % 10) == 0:
        print(x, m._objective(x)[0])
    logger.i+=1
logger.i = 0

m.optimize(method=tf.train.AdamOptimizer(), callback=logger)



[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 181586.72233875]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 180537.74127184]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 179500.65675795]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 178476.56694212]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 177466.08404523]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 176469.42592986]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 175486.5617425]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 174517.32678109]
Caught KeyboardInterrupt, setting model                  with most recent state.


In [24]:
y, _ = m.predict_y(X_train)
print("Train:", r_squared(y_train, y.flatten()))
y, _ = m.predict_y(X_test)
print("Test:", r_squared(y_test, y.flatten()))

Train: -4.42959616572
Test: -4.39652614283


In [4]:
import category_dae
import missForest as mf
import datasets

dataset = datasets.datasets(exclude_labels=False)["Soybean"]
info = category_dae.dataset_dimensions_info(dataset)
test_df, _ = mf.preprocess_dataframe(
    dataset[0], info, ignore_ordered=True)

+++ Importing BostonHousing
+++ Importing Soybean
+++ Importing Shuttle
+++ Importing Servo
+++ Importing LetterRecognition
+++ Importing BreastCancer
+++ Importing Ionosphere
V2 ; must have more than 1 possible value


In [9]:
y = test_df[info["cat_dummies"]["Class"]]
X = test_df.drop(y.keys(), axis=1)
X = (X-X.mean()) / X.std()
test_mask = np.random.rand(len(X)) < 0.2

In [55]:
n_features = X.shape[1]
n_classes = y.shape[1]
y_train = np.argmax(y.values, axis=1)[:, np.newaxis]
X_train = X.values

likelihood = GPy.likelihoods.MultiClass(n_classes)

kern = GPflow.kernels.Matern32(n_features) + GPflow.kernels.White(n_features)

m = GPflow.svgp.SVGP(X.values, ,
                   kern=kern, likelihood=likelihood,
                   Z=X.values[::5].copy(), num_latent=n_classes)

In [56]:
kern.matern32.lengthscales.fixed = True
kern.matern32.variance.fixed = True
kern.white.variance.fixed = True

In [57]:
def logger(x):
    if (logger.i % 10) == 0:
        print(x, m._objective(x)[0])
    logger.i+=1
logger.i = 0

m.optimize(method=tf.train.AdamOptimizer(), callback=logger)

InvalidArgumentError: Input matrix is not invertible.
	 [[Node: model.build_likelihood/conditional/MatrixTriangularSolve = MatrixTriangularSolve[T=DT_DOUBLE, adjoint=false, lower=true, _device="/job:localhost/replica:0/task:0/cpu:0"](model.build_likelihood/conditional/Cholesky, model.build_likelihood/conditional/model.kern.K/Add)]]

Caused by op 'model.build_likelihood/conditional/MatrixTriangularSolve', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/adria/venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/adria/venv/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/adria/venv/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
 

In [27]:
y_ = m.predict_f(X[test_mask])

In [28]:
y_

(array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]]),
 array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]]))